In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler

In [2]:
articles_df = pd.read_csv('shared_articles.csv')
interactions_df = pd.read_csv('users_interactions.csv')
articles_df.drop(['authorUserAgent', 'authorRegion', 'authorCountry'], axis=1, inplace=True)
interactions_df.drop(['userAgent', 'userRegion', 'userCountry'], axis=1, inplace=True)

In [3]:
articles_df.head(3)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [4]:
articles_df['eventType'].value_counts()

CONTENT SHARED     3047
CONTENT REMOVED      75
Name: eventType, dtype: int64

In [5]:
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.drop('eventType', axis=1, inplace=True)

In [6]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3047 entries, 1 to 3121
Data columns (total 9 columns):
timestamp          3047 non-null int64
contentId          3047 non-null int64
authorPersonId     3047 non-null int64
authorSessionId    3047 non-null int64
contentType        3047 non-null object
url                3047 non-null object
title              3047 non-null object
text               3047 non-null object
lang               3047 non-null object
dtypes: int64(4), object(5)
memory usage: 238.0+ KB


In [7]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72312 entries, 0 to 72311
Data columns (total 5 columns):
timestamp    72312 non-null int64
eventType    72312 non-null object
contentId    72312 non-null int64
personId     72312 non-null int64
sessionId    72312 non-null int64
dtypes: int64(4), object(1)
memory usage: 2.8+ MB


In [8]:
df = pd.merge(interactions_df[['contentId','personId', 'eventType']], articles_df[['contentId', 'title']], how = 'inner', on = 'contentId')

In [9]:
df.head(10)

,contentId,personId,eventType,title
0,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem
1,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem
2,-3499919498720038879,-108842214936804958,VIEW,Hiri wants to fix the workplace email problem
3,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem
4,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem
5,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem
6,-3499919498720038879,-8020832670974472349,VIEW,Hiri wants to fix the workplace email problem
7,-3499919498720038879,-8020832670974472349,VIEW,Hiri wants to fix the workplace email problem
8,-3499919498720038879,-9009798162809551896,LIKE,Hiri wants to fix the workplace email problem
9,-3499919498720038879,-9009798162809551896,VIEW,Hiri wants to fix the workplace email problem


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72269 entries, 0 to 72268
Data columns (total 4 columns):
contentId    72269 non-null int64
personId     72269 non-null int64
eventType    72269 non-null object
title        72269 non-null object
dtypes: int64(2), object(2)
memory usage: 2.8+ MB


In [11]:
df['eventType'].value_counts()

VIEW               61043
LIKE                5745
BOOKMARK            2463
COMMENT CREATED     1611
FOLLOW              1407
Name: eventType, dtype: int64

In [12]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 3.0, 
   'FOLLOW': 4.0,
   'COMMENT CREATED': 5.0,  
}

df['eventStrength'] = df['eventType'].apply(lambda x: event_type_strength[x])

In [13]:
df.head(10)

,contentId,personId,eventType,title,eventStrength
0,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem,1.0
1,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem,1.0
2,-3499919498720038879,-108842214936804958,VIEW,Hiri wants to fix the workplace email problem,1.0
3,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem,1.0
4,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem,1.0
5,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem,1.0
6,-3499919498720038879,-8020832670974472349,VIEW,Hiri wants to fix the workplace email problem,1.0
7,-3499919498720038879,-8020832670974472349,VIEW,Hiri wants to fix the workplace email problem,1.0
8,-3499919498720038879,-9009798162809551896,LIKE,Hiri wants to fix the workplace email problem,2.0
9,-3499919498720038879,-9009798162809551896,VIEW,Hiri wants to fix the workplace email problem,1.0


In [14]:
df = df.drop_duplicates()
grouped_df = df.groupby(['personId', 'contentId', 'title']).sum().reset_index()

In [15]:
grouped_df.sample(10)

,personId,contentId,title,eventStrength
29725,3636910968448833585,-7518373517401484139,My favorite people and resources to learn Andr...,1.0
7254,-5645906903825510952,-8370744479086515302,Mobile banking conquista o coração dos brasile...,1.0
33875,5497189205340943824,5037403311832115000,Cognizant Named a Digital Transformation Leade...,1.0
1764,-8674958742744576254,564282370384747453,Scrum Community - Scrum Alliance,1.0
22781,1097401149118884075,5313335392004163852,Salesforce Architect Journey,1.0
17344,-1257176162426022931,-8381230866408697127,Inteligência artificial da IBM criou 'trailer ...,15.0
37315,7554279798570266254,8712996026808754180,19 Tips For Everyday Git Use,1.0
27516,3106760029136205156,-8142426490949346803,A identidade visual de Budapeste para as Olimp...,3.0
39098,8516310373487915663,8095004770374551394,Microsoft Build: the 10 most important announc...,1.0
36389,6999578934585823267,-1021685224930603833,Indústria 4.0: desafios e oportunidades,1.0


In [16]:
grouped_df.dtypes

personId           int64
contentId          int64
title             object
eventStrength    float64
dtype: object

In [17]:
grouped_df['title'] = grouped_df['title'].astype("category")
grouped_df['personId'] = grouped_df['personId'].astype("category")
grouped_df['contentId'] = grouped_df['contentId'].astype("category")
grouped_df['person_id'] = grouped_df['personId'].cat.codes
grouped_df['content_id'] = grouped_df['contentId'].cat.codes

In [70]:
grouped_df.head(10)

,personId,contentId,title,eventStrength,person_id,content_id
0,-9223121837663643404,-8949113594875411859,"No Brasil, '25% dos celulares ainda são 'Burro...",1.0,0,65
1,-9223121837663643404,-8377626164558006982,Bad Writing Is Destroying Your Company's Produ...,1.0,0,159
2,-9223121837663643404,-8208801367848627943,Ray Kurzweil: The world isn't getting worse - ...,1.0,0,187
3,-9223121837663643404,-8187220755213888616,Organizing for digital acceleration: Making a ...,1.0,0,195
4,-9223121837663643404,-7423191370472335463,"Espresso Intents: não é magia, é tecnologia! -...",1.0,0,313
5,-9223121837663643404,-7331393944609614247,Here's proof that Google is getting serious ab...,1.0,0,327
6,-9223121837663643404,-6872546942144599345,My experience with Google's Associate Android ...,1.0,0,385
7,-9223121837663643404,-6728844082024523434,Seniority,1.0,0,416
8,-9223121837663643404,-6590819806697898649,Listas com RecyclerView - Android Dev BR,1.0,0,442
9,-9223121837663643404,-6558712014192834002,Google's fair use victory is good for open source,1.0,0,450


In [28]:
sparse_content_person = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['content_id'], grouped_df['person_id'])))
sparse_person_content = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['person_id'], grouped_df['content_id'])))

In [29]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

In [30]:
alpha = 15
data = (sparse_content_person * alpha).astype('double')

# Fit the model
model.fit(data)

100%|██████████| 50.0/50 [00:03<00:00, 15.36it/s]


In [55]:
content_id = 450
n_similar = 10

person_vecs = model.user_factors
content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))

scores = content_vecs.dot(content_vecs[content_id]) / content_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[content_id]), key=lambda x: -x[1])

In [56]:
for content in similar:
    idx, score = content
    print(grouped_df.title.loc[grouped_df.content_id == idx].iloc[0])

Google's fair use victory is good for open source
Up your DevOps chops with this online Kubernetes class
Google's Cloud Dataflow stomps on Apache Spark in new benchmark tests
Google lags behind Amazon and Microsoft's cloud in one important area
Meet Mycroft, the open source AI who wants to rival Siri, Cortana, and Alexa | ZDNet
Understanding User Psychology: Thinking Like a Game Designer
People dump AI advisors that give bad advice, while they forgive humans for doing the same
Google just open sourced something called 'Parsey McParseface,' and it could change AI forever
Google's AI engine is reading 2,865 romance novels to be more conversational
Deep learning software knows that a rose is a rose is a rosa rubiginosa


In [37]:
def recommend(person_id, sparse_person_content, person_vecs, content_vecs, num_contents=10):
    # Get the interactions scores from the sparse person content matrix
    person_interactions = sparse_person_content[person_id,:].toarray()
    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1
    # Make articles already interacted zero
    person_interactions[person_interactions > 1] = 0
    # Get dot product of person vector and all content vectors
    rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = person_interactions * rec_vector_scaled
    # Sort the indices of the content into order of best recommendations
    content_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    # Start empty list to store titles and scores
    titles = []
    scores = []

    for idx in content_idx:
        # Append titles and scores to the list
        titles.append(grouped_df.title.loc[grouped_df.content_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'title': titles, 'score': scores})

    return recommendations

In [60]:
# Get the trained person and content vectors. We convert them to csr matrices
person_vecs = sparse.csr_matrix(model.user_factors)
content_vecs = sparse.csr_matrix(model.item_factors)

# Create recommendations for person with id 50
person_id = 50

recommendations = recommend(person_id, sparse_person_content, person_vecs, content_vecs)

print(recommendations)

                                               title     score
0  Custo do Erro - Cinco motivos para investir em...  1.000000
1  Former Google career coach shares a visual tri...  0.889209
2  Ray Kurzweil: The world isn't getting worse - ...  0.816818
3           Drupal and ambitious digital experiences  0.810699
4                       Discutindo Devops na Prática  0.773286
5                            15 minutos sobre Docker  0.766407
6       10 Modern Software Over-Engineering Mistakes  0.760556
7  Desenvolvimento de Aplicativos Mobile com Xama...  0.720443
8                        Mastering Bash and Terminal  0.700380
9  How I built an app with 500,000 users in 5 day...  0.698131


In [40]:
grouped_df.loc[grouped_df['person_id'] == 50].sort_values(by=['eventStrength'], ascending=False)[['title', 'person_id', 'eventStrength']].head(10)

,title,person_id,eventStrength
1727,Acquia Engage 2016: Day One,50,3.0
1791,Um bilhão de arquivos mostram quem vence a dis...,50,3.0
1781,Acquia Engage Awards Finalists Announced,50,3.0
1778,Sharing innovation with your competitors - Dri...,50,3.0
1769,Don't document your code. Code your documentat...,50,3.0
1747,Who sponsors Drupal development? | Dries Buytaert,50,3.0
1768,Johnson & Johnson comprará grupo suíço por US$...,50,1.0
1767,Slack and Google announce partnership focused ...,50,1.0
1770,Rating the English Proficiency of Countries an...,50,1.0
1766,Infográfico: Algoritmos para Aprendizado de Má...,50,1.0


In [61]:
person_id = 2

recommendations = recommend(person_id, sparse_person_content, person_vecs, content_vecs)

print(recommendations)

                                               title     score
0                   Livro: Retrospectivas Divertidas  1.000000
1  Novo workaholic trabalha, pratica esportes e t...  0.823801
2          Google Ranking Factors: The Complete List  0.763991
3  Psicóloga de Harvard diz que as pessoas julgam...  0.672312
4                                          UX ou UI?  0.661945
5  ITA está oferecendo 10 cursos gratuitos a dist...  0.639597
6  How to Improve 8 Major Problem Areas for Japan...  0.635637
7              The brilliant mechanics of Pokémon Go  0.632819
8  Ray Kurzweil: The world isn't getting worse - ...  0.629383
9  'The Simpsons' celebrates 600 episodes with a ...  0.617249


In [59]:
grouped_df.loc[grouped_df['person_id'] == 2].sort_values(by=['eventStrength'], ascending=False)[['title', 'eventStrength', 'person_id']]

,title,eventStrength,person_id
51,Former Google career coach shares a visual tri...,6.0,2
48,Request lesson : How and when to use はず(=hazu)...,3.0,2
49,Aposta na inovação,3.0,2
50,"The Algorithm March, Japan's Strangely Enterta...",3.0,2
54,Como são escrita as risadas em japonês? - Suki...,3.0,2
52,A minha viagem à Maternidade #tetodomundo,1.0,2
53,Learn Hiragana: The Ultimate Guide,1.0,2


In [67]:
person_id = 1

recommendations = recommend(person_id, sparse_person_content, person_vecs, content_vecs)

print(recommendations)

                                               title     score
0  Como são escrita as risadas em japonês? - Suki...  0.891254
1                   Livro: Retrospectivas Divertidas  0.821983
2                                          UX ou UI?  0.814568
3  Request lesson : How and when to use はず(=hazu)...  0.771821
4                               Jenkins 2.0 is here!  0.724092
5  The Algorithm March, Japan's Strangely Enterta...  0.717467
6  'The Simpsons' celebrates 600 episodes with a ...  0.662476
7  ITA está oferecendo 10 cursos gratuitos a dist...  0.657344
8  How to Improve 8 Major Problem Areas for Japan...  0.652221
9                    40 Basic Japanese conversations  0.645782


In [69]:
grouped_df.loc[grouped_df['person_id'] == 1].sort_values(by=['eventStrength'], ascending=False)[['title', 'eventStrength', 'person_id']]

,title,eventStrength,person_id
44,Learn Hiragana: The Ultimate Guide,3.0,1
43,Firebase Test Lab for Android,1.0,1
45,"Fresco, sim! - Android Dev BR",1.0,1
46,Japanese for dummies,1.0,1
47,Firebase and Google Cloud: better together,1.0,1
